# Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import sklearn
import numpy as np

# parameters
input_channels = 5 # open, close, high, low, volume
activation_function = nn.ReLU()
learning_rate = 0.001
epochs = 1000
loss_function = nn.MSELoss()

# Data

In [ ]:
window_size = 5 # number of days we read at a time
steps = 100 # number of days to predict
df = pd.read_csv("../data.csv")
df = df[df["Company"] == "AAPL"] # select Apple's stock price data
close_data = list()
open_data = list()
high_data = list()
low_data = list()
volume_data = list()
data_target = list()

for i in range(len(df)):
  close_data.append(df["Close/Last"][i])
  open_data.append(df["Open"][i])
  high_data.append(df["High"][i])
  low_data.append(df["Low"][i])
  volume_data.append(df["Volume"][i]) # might need to normalize/standardize the volume data
  data_target.append(df["Close/Last"][i])

close_data.pop(0)
open_data.pop(0)
high_data.pop(0)
low_data.pop(0)
volume_data.pop(0)

close_data.reverse()
open_data.reverse()
high_data.reverse()
low_data.reverse()
volume_data.reverse()
data_target.reverse()

for i in range(window_size):
  data_target.pop(0)

data = list(zip(open_data, close_data, high_data, low_data, volume_data))
print(data)

[('$15.4779', '$15.4199', '$15.5311', '$15.3789', np.int64(218632537)), ('$15.4679', '$15.1768', '$15.4993', '$15.1554', np.int64(268548901)), ('$15.3379', '$15.2254', '$15.3482', '$15.1953', np.int64(207648981)), ('$15.2143', '$14.9639', '$15.2486', '$14.9539', np.int64(354477618)), ('$15.6761', '$15.7325', '$15.8782', '$15.545', np.int64(591624923)), ('$15.7393', '$15.6607', '$15.7643', '$15.5646', np.int64(229432412)), ('$15.5464', '$15.7496', '$15.7514', '$15.5121', np.int64(200082264)), ('$15.7429', '$15.9925', '$16.0711', '$15.7214', np.int64(248025441)), ('$16.07', '$16.19', '$16.3268', '$16.0439', np.int64(308960556)), ('$16.2496', '$16.1618', '$16.3337', '$16.0511', np.int64(322499391)), ('$16.2769', '$16.3099', '$16.3143', '$16.1879', np.int64(205544138)), ('$16.3575', '$16.5193', '$16.5304', '$16.3093', np.int64(273644955)), ('$16.5961', '$16.7661', '$16.8096', '$16.5054', np.int64(318310593)), ('$16.715', '$16.6161', '$16.8532', '$16.5061', np.int64(334812106)), ('$16.5643'

# Building the model
The network is set up to output a single value prediction. Probably the closing price for some future date.

In [ ]:
class Net(nn.Module): # the nn.Module is set up as the parent class for the Net Class
    def __init__(self):
        super(Net, self).__init__() # This calls the init method of the nn.Module parent class to ensure that its been initialized

        self.model = nn.Sequential(
            nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=2, stride=1),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=2, stride=1),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Linear(in_features=128, out_features=64),
            nn.Linear(64, 1)
        )

    def forward(self, x):

        return self.model(x)


net = Net()

Resources: https://www.geeksforgeeks.org/deep-learning/building-a-convolutional-neural-network-using-pytorch/


# Training the model

In [ ]:
def train(model: nn.Module, train_x, train_y, epochs, learning_rate):
  optimizer = optim.Adam(net.parmeters(), lr=learning_rate)

  for epoch in range(epochs):
    y_hat = net(train_x)
    loss = loss_function(y_hat, train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0: # print out the loss as the model is training
      print(f"Epoch: {epoch}, Loss: {loss.item():.3f}")

# train(net, train_x, train_y, epochs, learning_rate)


# Running the model

In [ ]:
# torch.save(net, "cnn.pt")